# The Art of Context Reduction: The Summarizer Agent

Copyright 2025, Denis Rothman

**Goal**:
* Managing API costs and token limits efficiently.    
•	Implementing the count_tokens Utility for proactive cost management.    
•	Building the agent_summarizer to act as an intelligent gatekeeper for large contexts.   
•	Integrating the new agent into the Context Engine's dynamic planning workflow.

**Note on upgrade comments:** The comment *New* in the code is related to the upgrades in made up to and in Chapter 5.

Starting with sthis chapter the files in `commons` will be in new subdirectory of commons. For example for this chapter the `commons` files will be in `commons/chc6/`

**Upgrades in this Chapter described in the book**:

*   **Phase 1**: Verify the `count_tokens` Utility

*   **Phase 2**: Create the New `agent_summarizer` by adding a summarizing agent in the `agents.py` file.

*   **Phase 3**: Register the Summarizer Agent by updating the AgentRegistry in `registry.py`.

*   **Phase 4**: Verify the Core Engine (no changes)

*   **Phase 5**: Run Context Engine
    
    1.Run Control Deck call of Chapter 5 for a backward compatibility check

    2.Run Control Deck example Chapter 6 with new goal and cell














# I. Inititalization

## GitHub

In [ ]:
print("Downloading files from public repository...")

# The -f flag tells curl to fail on an error (like 404)
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch6/utils.py --output utils.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch6/helpers.py --output helpers.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch6/agents.py --output agents.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch6/registry.py --output registry.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch6/engine.py --output engine.py
# (You might want to add a check here to see if the files actually exist now)
print("✅ File download attempt finished!")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1825  100  1825    0     0   9919      0 --:--:-- --:--:-- --:--:--  9972
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4302  100  4302    0     0  24276      0 --:--:-- --:--:-- --:--:-- 24305
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  7202  100  7202    0     0  38585      0 --:--:-- --:--:-- --:--:-- 38720
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0    14    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (22) The requested URL returned error: 404
  %

## Installation and client setup

In [ ]:
#Installation and Client Setup

# Import the setup functions from your new utility file
import utils

# Run the installation
utils.install_dependencies()

# Initialize the OpenAI and Pinecone clients
client, pc = utils.initialize_clients()

🚀 Installing required packages...
✅ All packages installed successfully.

🔑 Initializing API clients...
   - OpenAI client initialized.
   - Pinecone client initialized.
✅ Clients initialized successfully.


## Context Engine library Import

In [ ]:
# 1. Import the hardened helper functions (LLM, Embeddings, Pinecone)
import helpers

# 2. Import the specialist agent functions (Librarian, Researcher, Writer)
import agents

# 3. Import the AGENT_TOOLKIT object that knows about all the agents
from registry import AGENT_TOOLKIT

# 4. Import the main context_engine function that orchestrates the entire process
from engine import context_engine

## Engine Room

In [ ]:
# === ENGINE ROOM: The Main Execution Function ===
# This function contains all the logic to run the engine.
# We define it here so our final cell can be very simple.

import logging
import pprint
from IPython.display import display, Markdown

def execute_and_display(goal, config, client, pc):
    """
    Runs the context engine with a given goal and configuration,
    then displays the final output and the technical trace.
    """
    logging.info(f"******** Starting Engine for Goal: '{goal}' **********\n")

    # 1. Run the Context Engine using the provided configuration
    result, trace = context_engine(
        goal,
        client=client,
        pc=pc,
        **config  # Unpack the config dictionary into keyword arguments
    )

    # 2. Display the Final Result for the main reader
    print("--- FINAL OUTPUT ---")
    if result:
        display(Markdown(result))
    else:
        print(f"The engine failed to produce a result. Status: {trace.status}")

    # 3. Display the Technical Trace for the developer/technical reader
    print("\n\n--- TECHNICAL TRACE (for the tech reader) ---")
    if trace:
        print(f"Trace Status: {trace.status}")
        print(f"Total Duration: {trace.duration:.2f} seconds")
        print("Execution Steps:")
        # Use pprint for a clean, readable dictionary output
        pp = pprint.PrettyPrinter(indent=2)
        pp.pprint(trace.steps)

## Control Deck configuration

In [ ]:
# 1. Define all configuration variables for this run in a dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5",
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

#II.CONTROL DECK

=== CONTROL DECK: Define Goal and Run Engine ===
This is the main interactive cell.
1. Change the 'goal' variable to your desired task.
2. Run this cell.


In [ ]:
# === CONTROL DECK: Context Reduction Workflow ===

# 1. Define a large piece of text that would be expensive or too long
# to use as direct context for the Writer agent.
large_text_from_researcher = """
Juno is a NASA space probe orbiting the planet Jupiter. It was launched from Cape Canaveral Air Force Station on August 5, 2011, as part of the New Frontiers program. Juno entered a polar orbit of Jupiter on July 5, 2016, to begin a scientific investigation of the planet. After completing its primary mission, it received a mission extension. Juno's mission is to measure Jupiter's composition, gravitational field, magnetic field, and polar magnetosphere. It is also searching for clues about how the planet formed, including whether it has a rocky core, the amount of water present within the deep atmosphere, mass distribution, and its deep winds, which can reach speeds up to 618 kilometers per hour (384 mph). Juno is the second spacecraft to orbit Jupiter, after the nuclear-powered Galileo orbiter, which orbited from 1995 to 2003. Unlike all earlier spacecraft to the outer planets, Juno is powered by solar arrays, which are commonly used by satellites orbiting Earth and working in the inner Solar System, whereas radioisotope thermoelectric generators are commonly used for missions to the outer Solar System and beyond. For Juno, however, the three largest solar array wings ever deployed on a planetary probe play an integral role in stabilizing the spacecraft and generating power.
"""

# 2. Define a goal that requires both using the large text AND a creative step.
# This forces the Planner to recognize the need for summarization before writing.
goal = f"""First, summarize the following text about the Juno probe to extract only the key facts about its scientific mission and instruments. Then, using that summary, write a short, suspenseful scene for a children's story about the probe's dangerous arrival at Jupiter.

--- TEXT TO USE ---
{large_text_from_researcher}
"""

# 3. Use the same configuration dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 4. Call the execution function
execute_and_display(goal, config, client, pc)

--- FINAL OUTPUT ---


Mission snapshot: Exploring Jupiter from pole to core

This mission is designed to reveal how Jupiter formed and what lies beneath its cloud tops. Flying a polar orbit, the spacecraft investigates the planet’s interior structure, determines whether a rocky core exists, quantifies water in the deep atmosphere, maps mass distribution, characterizes deep winds, and studies Jupiter’s composition, gravity, magnetic field, and polar magnetosphere.

Key measurements:
- Atmospheric composition
- Gravitational field
- Magnetic field
- Polar magnetosphere
- Deep atmospheric water abundance
- Interior mass distribution
- Deep wind speeds (up to ~618 km/h)

Unique engineering:
- Polar orbit around Jupiter to directly sample the polar magnetosphere and global fields
- Solar-powered operation in the outer Solar System using three very large solar array wings (instead of RTGs)
- The arrays are the largest ever flown on a planetary probe and also assist with spacecraft stabilization

Together, these investigations connect Jupiter’s present-day structure and dynamics to its formation, offering a clearer picture of how giant planets—and planetary systems—take shape.



--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 62.80 seconds
Execution Steps:
[ { 'agent': 'Summarizer',
    'output': { 'summary': '- Mission goals: Investigate Jupiter’s formation '
                           'and interior structure; determine if it has a '
                           'rocky core; quantify water in the deep atmosphere; '
                           'map mass distribution; characterize deep winds; '
                           'study composition, gravity, magnetic field, and '
                           'polar magnetosphere.\n'
                           '- Key measurements: Composition; gravitational '
                           'field; magnetic field; polar magnetosphere; deep '
                           'atmospheric water; mass distribution; deep wind '
                           'speeds (up to ~618 km/h).\n'
                           '- Power source: Solar-powered via three very large '
                           'solar array 